## 에이전트 대화 저장 및 재개

이 과정에서는 에이전트 대화(AgentThread)를 저장소에 저장하고 나중에 다시 불러오는 방법을 보여줍니다.

에이전트를 서비스 또는 클라이언트 애플리케이션에서 호스팅할 때, 여러 요청이나 세션에 걸쳐 대화 상태를 유지해야 하는 경우가 많습니다. AgentThread를 저장하면 대화 컨텍스트를 나중에 다시 불러올 수 있습니다.

#### 에이전트 생성 및 쓰레드 시작

에이전트를 생성하고 대화 상태를 저장할 새 스레드를 만듭니다.

In [ ]:
from azure.identity import AzureCliCredential
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient

agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
    name="Assistant",
    instructions="You are a helpful assistant who responds in korean."
)

thread = agent.get_new_thread()

에이전트를 실행할 때, 스레드를 전달하여 AgentThread가 이러한 대화내용을 포함하게 합니다.

In [ ]:
# Run the agent and append the exchange to the thread
response = await agent.run("Tell me a short pirate joke.", thread=thread)
print(response.text)

#### 쓰레드를 저장소에 저장하기

해당 스레드에서 serialize 메서드를 호출하여 데이터를 딕셔너리로 직렬화합니다. 그런 다음, JSON으로 변환하여 데이터베이스, Blob 스토리지 또는 파일에 저장할 수 있습니다.

In [ ]:
import json
import tempfile
import os

# Serialize the thread state
serialized_thread = await thread.serialize()
serialized_json = json.dumps(serialized_thread)

# 예제 : 현재 경로에 agent_thread.json 파일로 저장하기(replace with DB or blob storage in production)
current_dir = os.getcwd()
# temp_dir = tempfile.gettempdir()
print(f"Current working directory: {current_dir}")

file_path = os.path.join(current_dir, "agent_thread.json")
with open(file_path, "w") as f:
    f.write(serialized_json)

#### 쓰레드를 저장소에 불러오기

저장소에 저장된 대화를 다시 불러오려면, 저장소에 저장된 JSON 데이터를 불러와서 AgentThread 인스턴스를 다시 생성하면 됩니다. 이 경우, 스레드는 에이전트 인스턴스를 사용하여 역직렬화 되어야 하며, 사용되는 에이전트 유형은 원래 스레드를 생성할 때 사용한 것과 동일해야 합니다. 에이전트마다 고유한 스레드 유형이 있을 수 있으며, 해당 에이전트 유형에 특화된 추가 기능을 사용하여 스레드를 생성할 수 있기 때문입니다.

In [ ]:
# Read persisted JSON
with open(file_path, "r") as f:
    loaded_json = f.read()

reloaded_data = json.loads(loaded_json)

# Deserialize the thread into an AgentThread tied to the same agent type
resumed_thread = await agent.deserialize_thread(reloaded_data)

가져온 스레드를 사용하여 대화를 다시 이어갈 수 있습니다.

In [ ]:
# Continue the conversation with resumed thread
response = await agent.run("Now tell that joke in the voice of a pirate.", thread=resumed_thread)
print(response.text)

#### 실용적인 멀티턴 예제

다음은 여러 상호작용에 걸쳐 컨텍스트를 유지하는 방법을 보여주는 완전한 예제입니다:

In [5]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity import AzureCliCredential

agent = ChatAgent(
    chat_client=AzureAIAgentClient(credential=AzureCliCredential()),
    name="Assistant",
    instructions="You are a helpful assistant who responds in korean."
)

# Create a thread for persistent conversation
thread = agent.get_new_thread()

# First interaction
response1 = await agent.run("My name is Alice", thread=thread)
print(f"Agent: {response1.text}")

# Second interaction - agent remembers the name
response2 = await agent.run("What's my name?", thread=thread)
print(f"Agent: {response2.text}")  # Should mention "Alice"

# Serialize thread for storage
serialized = await thread.serialize()

# Later, deserialize and continue conversation
new_thread = await agent.deserialize_thread(serialized)
response3 = await agent.run("What did we talk about?", thread=new_thread)
print(f"Agent: {response3.text}")  # Should remember previous context


Agent: Hello Alice! How can I assist you today? 😊
Agent: Your name is Alice! How can I help you today? 😊
Agent: So far, you told me your name is Alice. You also asked me what your name is, and then you asked what we talked about. If you have more questions or need help with something specific, just let me know!
